# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
##### Your Code Here #####
import numpy as np

X = np.array([[0.,0.,1.],
               [0.,1.,1.],
               [1.,0.,1.],
               [0.,1.,0.],
               [1.,0.,0.],
               [1.,1.,1.],
               [0.,0.,0.]])

y = np.array([[0.],
              [1.],
              [1.],
              [1.],
              [1.],
              [0.],
              [0.]])

In [2]:
class Neural_Network(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

    def sigmoid(self, s):
        return 1/(1+np.exp(-s))

General Output:

In [3]:
# General output
NN = Neural_Network()
output = NN.feed_forward(X)
print("Output:\n", output)

Output:
 [[0.27508871]
 [0.37592531]
 [0.39622093]
 [0.35895832]
 [0.37572934]
 [0.46772734]
 [0.25427298]]


Error with General Output:

In [4]:
# expected - predicted
print("Error:\n",y - output)

Error:
 [[-0.27508871]
 [ 0.62407469]
 [ 0.60377907]
 [ 0.64104168]
 [ 0.62427066]
 [-0.46772734]
 [-0.25427298]]


## Backpropagation (Assigning blame for bad predictions and delivering justice - repeatedly and a little bit at a time)

What in our model could be causing our predictions to suck so bad? 

Well, we know that our inputs (X) and outputs (y) are correct, if they weren't then we would have bigger problems than understanding backpropagation.

We also know that our activation function (sigmoid) is working correctly. It can't be blamed because it just does whatever we tell it to and transforms the data in a known way.

So what are the potential culprits for these terrible predictions? The **weights** of our model. Here's the problem though. I have weights that exist in both layers of my model. How do I know if the weights in the first layer are to blame, or the second layer, or both? 

Lets investigate. And see if we can just eyeball what should be updated.

In [5]:
class Neural_Network(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
   
    # sigmoidPrime: derivative of sigmoid function 
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    # backpropagation function
    # backward propagate through the network
    def backward(self, X, y, output):
        # error in output for every prediction
        self.output_error = y - output
        # applying derivative of sigmoid to error
        # delta: how much I want to change all my weights 
        # given my error multiplied by derivative sigmoid of output
        self.output_delta = self.output_error * self.sigmoidPrime(output)
        
        # z2 error: how much our hidden layer weights (2nd to last layer) 
        # contributed to output error
        self.z2_error = self.output_delta.dot(self.L2_weights.T)
        # applying derivative of sigmoid to z2 error
        # z2_delta: how much weights in this layer will change based on error calculated
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Here: updating the 2 sets of weights 
        # (between input & hidden AND between hidden & output)
        # L1_weights: adjusting first set (input --> hidden) weights
        self.L1_weights += X.T.dot(self.z2_delta)
        # L2_weights: adjusting second set (hidden --> output) weights
        self.L2_weights += self.activated_hidden.T.dot(self.output_delta)
    
    # Telling Network to run certain # of times
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [6]:
# Instantiating Neural_Network object
# Running it 1000 times
# And Printing out every 50th Epoch
NN = Neural_Network()
for i in range(1000): # trains the NN 1,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        print("Input: \n", X) 
        print("Actual Output: \n", y)  
        print("Predicted Output: \n" + str(NN.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
        print("\n")
    NN.train(X, y)

+---------- EPOCH 1 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.41889582]
 [0.50553825]
 [0.48411313]
 [0.40945109]
 [0.36914483]
 [0.54337666]
 [0.30071468]]
Loss: 
0.2597884525694637


+---------- EPOCH 2 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.45912393]
 [0.54887971]
 [0.53192677]
 [0.45947723]
 [0.42397018]
 [0.59050771]
 [0.34195413]]
Loss: 
0.2461434355707411


+---------- EPOCH 3 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.48489247]
 [0.57857705]
 [0.56480307]
 [0.49656253]
 [0.46518465]
 [0.62350879]
 [0.37153598]]
Loss: 
0.2383419028813026


+

In [7]:
##### Your Code Here #####
import numpy as np

X = np.array([[0.,0.,1.],
               [0.,1.,1.],
               [1.,0.,1.],
               [0.,1.,0.],
               [1.,0.,0.],
               [1.,1.,1.],
               [0.,0.,0.]])

y = np.array([[0.],
              [1.],
              [1.],
              [1.],
              [1.],
              [0.],
              [0.]])

In [8]:
class Neural_Network_ReLU(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.ReLU(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.ReLU(self.output_sum)
        return self.activated_output

    def ReLU(self, s):
        return s * (s > 0)
  
    def ReLUPrime(self, s):
        s_copy = s
        s_copy[s_copy<=0] = 0
        s_copy[s_copy>0] = 1
        return s_copy
    
    # backpropagation function
    # backward propagate through the network
    def backward(self, X, y, output):
        # error in output for every prediction
        self.output_error = y - output
        # applying derivative of ReLU to error
        # delta: how much I want to change all my weights 
        # given my error multiplied by derivative ReLU of output
        self.output_delta = self.output_error * self.ReLUPrime(output)
        
        # z2 error: how much our hidden layer weights (2nd to last layer) 
        # contributed to output error
        self.z2_error = self.output_delta.dot(self.L2_weights.T)
        # applying derivative of ReLU to z2 error
        # z2_delta: how much weights in this layer will change based on error calculated
        self.z2_delta = self.z2_error * self.ReLUPrime(self.activated_hidden)
        
        # Here: updating the 2 sets of weights 
        # (between input & hidden AND between hidden & output)
        # L1_weights: adjusting first set (input --> hidden) weights
        self.L1_weights += X.T.dot(self.z2_delta)
        # L2_weights: adjusting second set (hidden --> output) weights
        self.L2_weights += self.activated_hidden.T.dot(self.output_delta)
    
    # Telling Network to run certain # of times
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

Somehow this never converged using ReLU function!!! Don't know why... 

In [9]:
NN = Neural_Network_ReLU()
for i in range(10000): # trains the NN 10,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 1000 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        print("Input: \n", X) 
        print("Actual Output: \n", y)  
        print("Predicted Output: \n" + str(NN.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
        print("\n")
    NN.train(X, y)

+---------- EPOCH 1 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 2 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 3 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 4 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [1]:
from mlxtend.data import mnist_data
X, y = mnist_data()

In [2]:
import numpy as np

In [3]:
X.shape, y.shape

((5000, 784), (5000,))

In [8]:
y = np.reshape(y,(-1, 1))
y.shape

(5000, 1)

In [4]:
X[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,  51., 159., 253., 159.,  50.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [14]:
# dataset = np.concatenate((X, y[:, None]), 1)
# np.savetxt("minst.csv", dataset, delimiter=",")

In [5]:
class Neural_Network2(object):
    def __init__(self):
        self.inputs = 784
        self.hiddenNodes = 800
        self.outputNodes =10
        
        # Initialize weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) #(2x3)
#       self.L2_weights = np.random.randn(self.hiddenNodes, self.hiddenNodes)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations on weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def backward(self, X, y, o):
        #backward propgate through the network
        self.o_error = y - o # error in output
        self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid ot error
        
        self.z2_error = self.o_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) # applying derivative of sigmoid to z2 error
        
        self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
        self.L2_weights += self.activated_hidden.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [9]:
NN2 = Neural_Network2()
for i in range(1000): # trains the NN 1,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('+----------EPOCH', i+1, '----------+')
        print("Input: \n", X)
        print("Actual Output: \n", y)
        print("Predicted Output: \n" + str(NN2.feed_forward(X)))
        print("Loss: \n" + str(np.mean(np.square(y - NN2.feed_forward(X))))) #mean sum squared loss
        print("\n")
    NN2.train(X,y)

+----------EPOCH 1 ----------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0]
 [0]
 [0]
 ...
 [9]
 [9]
 [9]]
Predicted Output: 
[[9.99999997e-01 8.39736312e-03 2.64522694e-10 ... 5.14790598e-15
  9.94625400e-01 7.55355645e-01]
 [9.99999754e-01 9.74089607e-01 3.66435438e-11 ... 1.26198832e-07
  2.76280552e-08 9.98671508e-01]
 [3.72813989e-01 9.99995941e-01 1.49276081e-11 ... 1.12284459e-11
  9.20175787e-02 1.16354183e-02]
 ...
 [8.49642306e-03 1.00000000e+00 2.97810807e-23 ... 1.23603468e-17
  9.99998619e-01 9.99981996e-01]
 [1.96468640e-01 4.72353540e-06 7.61870906e-05 ... 4.73734381e-14
  1.00000000e+00 1.02238591e-03]
 [1.00000000e+00 9.99999996e-01 6.48787520e-14 ... 8.52527051e-13
  9.99999998e-01 6.80166960e-01]]
Loss: 
24.856036424696743


+----------EPOCH 2 ----------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

Loss: 
20.5


+----------EPOCH 700 ----------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0]
 [0]
 [0]
 ...
 [9]
 [9]
 [9]]
Predicted Output: 
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
Loss: 
20.5


+----------EPOCH 750 ----------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0]
 [0]
 [0]
 ...
 [9]
 [9]
 [9]]
Predicted Output: 
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
Loss: 
20.5


+----------EPOCH 800 ----------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

In [13]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?